In [1]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [2]:
def gerar_rede(num_classes: int, n_neurons: int, n_hidden_layers: int):
    dir_path = os.getcwd()
    
    metrics_file = f"{dir_path}/training_metrics.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Model Name", "Training Time", "Train Accuracy", "Test Accuracy"])

    file_data_train = os.path.join(dir_path, "data_train.csv")
    file_data_test = os.path.join(dir_path, "data_test.csv")
    
    data_train = pd.read_csv(file_data_train).to_numpy()
    data_test = pd.read_csv(file_data_test).to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    extentions = ['h5', 'keras']
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons"
    )
    if not os.path.join(dir_path, "models"):
        os.makedirs(os.path.join(dir_path, "models"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}.{extentions[1]}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(f'{model_path}.{extentions[1]}')

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    # model.evaluate(x_test, y_test_ohe, verbose=2)
    model.save(f'{model_path}.{extentions[0]}')
    
    train_loss, train_acc = model.evaluate(x_train, y_train_ohe, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test_ohe, verbose=2)
    
    metrics_df.loc[len(metrics_df)] = { # type: ignore
        "Model Name": f"model_{n_hidden_layers}layers_{n_neurons}neurons",
        "Training Time": end - start,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    } 
    metrics_df.to_csv(metrics_file, index=False)

In [7]:
n_hidden_layers = 5
n_neurons = 20
num_classes = 2
gerar_rede(num_classes, n_neurons, n_hidden_layers)

Epoch 1/100
114/114 - 4s - 38ms/step - accuracy: 0.8088 - loss: 0.5069 - val_accuracy: 0.8947 - val_loss: 0.2399
Epoch 2/100
114/114 - 1s - 6ms/step - accuracy: 0.9253 - loss: 0.1906 - val_accuracy: 0.9561 - val_loss: 0.1136
Epoch 3/100
114/114 - 1s - 7ms/step - accuracy: 0.9516 - loss: 0.1280 - val_accuracy: 0.9649 - val_loss: 0.0932
Epoch 4/100
114/114 - 1s - 5ms/step - accuracy: 0.9341 - loss: 0.1507 - val_accuracy: 0.9649 - val_loss: 0.0995
Epoch 5/100
114/114 - 0s - 3ms/step - accuracy: 0.9648 - loss: 0.0979 - val_accuracy: 0.9649 - val_loss: 0.1037
Epoch 6/100
114/114 - 0s - 3ms/step - accuracy: 0.9582 - loss: 0.1161 - val_accuracy: 0.9649 - val_loss: 0.1151
Epoch 7/100
114/114 - 0s - 3ms/step - accuracy: 0.9736 - loss: 0.0787 - val_accuracy: 0.9561 - val_loss: 0.1538
Epoch 8/100
114/114 - 0s - 3ms/step - accuracy: 0.9692 - loss: 0.0846 - val_accuracy: 0.9474 - val_loss: 0.1520
Epoch 9/100
114/114 - 0s - 4ms/step - accuracy: 0.9736 - loss: 0.0789 - val_accuracy: 0.9561 - val_loss

Resultado Teste
15/15 - 0s - 6ms/step - accuracy: 0.9824 - loss: 0.0519
4/4 - 0s - 18ms/step - accuracy: 0.9825 - loss: 0.0678
